<a href="https://colab.research.google.com/github/dyutikandala/ME36_Miniproject1/blob/main/ME36_Mini_Project1_Dyutikandala_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:

# ME36 - Mini Project 1 - A Comprehensive ETL Workflow with Python for Data Engineers.

# Problem statement: This project highlights the practical implementation of ETL processes using Python.
# This involves data extraction from multiple file formats, transformation of units, and loading of the final data into a structured CSV format.
# Demonstrate essential data engineering skills. Additionally, by logging each step of the process you can monitor the progress and debug issues if they arise.

# Importing required libraries.

import glob                         # this helps to handle different file formats.
import pandas as pd                 # this helps to read CSV and JSON files.
import xml.etree.ElementTree as ET  # this helps to parse XML data.
from datetime import datetime       # this helps to track the progress of each phase through logging.
import os

# Setting up file names to store logs and transformed data in current working directory.

timestamp_suffix = datetime.now().strftime('%Y%m%d_%H%M%S')
logfile    = "logfile_"+timestamp_suffix+".txt"            # All ETL event logs will be recorded into this file.
targetfile = "transformed_data.csv"   # Target file to store transformed data.

#Removing source files from previous run, if any
for f in glob.glob('source*'):
  os.remove(f)

#Remove target file from previous run, if any.
if os.path.exists(targetfile):
  os.remove(targetfile)

# Downloading the source dataset in .zip, containing multiple file formats from given URL into current working directory.
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

# Unzipping all source files into current working directory.
!unzip source.zip

# ****************************************** EXTRACT PHASE ********************************************************

# Extract Approach:Three different functions to extract data from CSV, JSON, and XML files respectively.
#                  Then a master function to combine all files into one single data frame.

# CSV Extract Function
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

# JSON Extract Function
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

# XML Extract Function
def extract_from_xml(file_to_process):
    #dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    rows=[]
    for person in root:                               # Looping for each element i.e. each person in xml.
        name = person.find("name").text               # Storing value from name field for the iteration.
        height = float(person.find("height").text)    # Storing value from height field value for the iteration.
        weight = float(person.find("weight").text)    # Storing value from weight field value for the iteration.

        #dataframe = pd.concat([dataframe,pd.DataFrame({"name":name, "height":height, "weight":weight},ignore_index=True)],ignore_index=True)
        rows.append({"name":name, "height":height, "weight":weight}) # storing data into the master dataframe for the iteration

    dataframe = pd.DataFrame(rows,columns=["name", "height", "weight"])
    return dataframe

# A master function to combine the extracted data from multiple files into a single DataFrame.
def extract():
    # creating an empty data frame to hold extracted data.
    extracted_data = pd.DataFrame(columns=['name','height','weight'])

    # Finding CSV files and writing data into the above created dataframe.
    for csvfile in glob.glob("*.csv"):    # finding .csv files in current directory
        extracted_data = pd.concat([extracted_data,extract_from_csv(csvfile)], ignore_index=True)

    # Finding JSON files and writing data into the above created dataframe.
    for jsonfile in glob.glob("*.json"):   # finding .json files in current directory
        extracted_data = pd.concat([extracted_data,extract_from_json(jsonfile)], ignore_index=True)

    # Finding  XML files and writing data into the above created dataframe.
    for xmlfile in glob.glob("*.xml"):     # finding . xml files in current directory
        extracted_data = pd.concat([extracted_data,extract_from_xml(xmlfile)], ignore_index=True)

    # Data cleansing i.e. removing duplicate data if required.
    # extracted_data = extracted_data.drop_duplicates()

    return extracted_data

# ****************************************** TRANSFORM PHASE ********************************************************

# Transformations required: Convert height from inches to meters and weight from pounds to kilograms.
# Transform approach: A function to perform arthemetic operations convert height and weight fields as per appropriate converstion formule.
#                     and replacing the old value with converted value in the dataframe.

def transform(data):

        #Converting height from inches to meters and rounding off to two decimals(Conversion formula: one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)

        #Converting  weight from pounds to kilograms and round off to two decimals(Conversion formula: one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)

        return data  # Returning dataframe with converted data.

# ***************************************  LOAD PHASE *************************************************************

# Function for Loading extracted and transformed data set into a target .csv file, which can later be loaded into a relational database or used for further processing.

def load(targetfile,data_to_load):

    data_to_load.to_csv(targetfile) #Loading transformed data into Target file.

# ************************************ LOGGING **************************************************************

# A function to log each ETL phase (Extraction, Transformation, Loading) with a timestamp to ensure traceability and monitoring.
# Logs are generated with timestamp and stage of each phase, written to text file for auditing or troubleshooting purposes.

def log(message):

    timestamp_format = '%Y-%h-%d %H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format) # Convert timestamp to string

    with open(logfile,"a") as f:  # creates a log.txt file if None
        f.write(timestamp + ',' + message + '\n')


# *********************************** EXECUTION ************************************************

# The ETL process follows the following sequence - Extract phase -> Transform phase -> Load phase.
# Logging happens at the start and end of each phase.

log("ETL process Started.")        # Calling log() function to log ETL job start.

log("Extract phase Started.")      # Calling log() function to log beginning of Extract phase.
extracted_data = extract()         # Calling extract() master function, returns "extracted_data" from all source files in single dataframe.
log("Extract phase Ended.")        # Calling log() function to log ending of Extract phase.

log("Transform phase Started.")                 # Calling log() function to log beginning of Transform phase.
transformed_data = transform(extracted_data)    # Calling transform() function with "extracted_data" dataframe as input. Returns the same dataframe with converted values for height and weight fields.
log("Transform phase Ended.")                   # Calling log() function to log ending of transform phase.

log("Load phase Started.")                      # Calling log() function to log beginning of load phase.
load(targetfile,transformed_data)               # Calling load() function with transformed_data and .CSV target file as input.
log("Load phase Ended.")                        # Calling log() function to log ending of load phase.

log("ETL Process Ended")             # Calling log() function to log ETL job end.



--2024-12-31 13:43:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2024-12-31 13:43:52 (666 MB/s) - ‘source.zip’ saved [2707/2707]

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  infla

<ipython-input-61-58f8e3882f56>:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data,extract_from_csv(csvfile)], ignore_index=True)
